In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load your dataset and model here (replace with your actual dataset)
df = pd.read_csv("Final_File.csv")
X = df[['B10_AVG_BATTERY_TEMP_PS_10B', 'B10_QUENCH_DUR_SEC_x', 'B10_TIME_TO_REACH_PEAK_POINT_x', 'B11_AVG_CHG_FORCE_y', 'B11_DURATION_EMPTY_OVENS_y', 'Blend_MAX_EXPANSION']]
y = df['AMS_IBF']

# Initialize the RandomForestRegressor model
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X, y)

# Function to categorize AMS value
def get_ams_status(ams_value):
    if ams_value <= 42.5:
        return "BAD AMS"
    elif 42.5 < ams_value <= 43:
        return "MID AMS"
    else:
        return "HIGH AMS"

# Function to predict AMS value
def predict_ams(temp, quench_dur, time_to_peak, avg_chg_force, empty_ovens_dur, blend_expansion):
    input_data = np.array([[temp, quench_dur, time_to_peak, avg_chg_force, empty_ovens_dur, blend_expansion]])
    predicted_ams = rf_regressor.predict(input_data)[0]
    return predicted_ams

# Create input widgets for each feature
temp_input = widgets.FloatText(value=0.0, description='B10_AVG_BATTERY_TEMP_PS_10B:', style={'description_width': 'initial'}, layout={'width': '300px'})
quench_dur_input = widgets.FloatText(value=0.0, description='B10_QUENCH_DUR_SEC_x:', style={'description_width': 'initial'}, layout={'width': '300px'})
time_to_peak_input = widgets.FloatText(value=0.0, description='B10_TIME_TO_REACH_PEAK_POINT_x:', style={'description_width': 'initial'}, layout={'width': '300px'})
avg_chg_force_input = widgets.FloatText(value=0.0, description='B11_AVG_CHG_FORCE_y:', style={'description_width': 'initial'}, layout={'width': '300px'})
empty_ovens_dur_input = widgets.FloatText(value=0.0, description='B11_DURATION_EMPTY_OVENS_y:', style={'description_width': 'initial'}, layout={'width': '300px'})
blend_expansion_input = widgets.FloatText(value=0.0, description='Blend_MAX_EXPANSION:', style={'description_width': 'initial'}, layout={'width': '300px'})

# Output widget for displaying predictions
output = widgets.Output()

# Button widget to trigger prediction
predict_button = widgets.Button(description="Predict AMS", button_style='info', tooltip='Click to predict AMS', layout={'width': '200px'})

# Function to handle button click event
def on_button_clicked(b):
    with output:
        clear_output()
        try:
            # Validate input values
            if any(v == 0 for v in [temp_input.value, quench_dur_input.value, time_to_peak_input.value, avg_chg_force_input.value, empty_ovens_dur_input.value, blend_expansion_input.value]):
                print("Error: All feature values must be non-zero.")
                return
            
            # Predict AMS value
            predicted_ams = predict_ams(temp_input.value, quench_dur_input.value, time_to_peak_input.value, avg_chg_force_input.value, empty_ovens_dur_input.value, blend_expansion_input.value)
            
            # Get AMS status
            ams_status = get_ams_status(predicted_ams)
            
            # Display predicted AMS and status
            print(f'Predicted AMS: {predicted_ams:.2f}')
            print(f'AMS Status: {ams_status}')
        except ValueError:
            print("Please enter valid numerical values for all fields.")

# Bind button click event to handler function
predict_button.on_click(on_button_clicked)

# Arrange widgets using VBox and display them
inputs = widgets.VBox([temp_input, quench_dur_input, time_to_peak_input, avg_chg_force_input, empty_ovens_dur_input, blend_expansion_input])
display(widgets.VBox([inputs, widgets.HBox([predict_button]), output]))
